In [1]:

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, pipeline, metrics, grid_search


/Users/ankitgupta/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/ankitgupta/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
air_visit_data = pd.read_csv('air_visit_data.csv')
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['air_store_id'] = sample_submission['id'].apply(lambda x: x.split('_')[0]+str('_') +x.split('_')[1])
sample_submission['visit_date'] = sample_submission['id'].apply(lambda x: x.split('_')[2])
sample_submission['visit_date'] = pd.to_datetime(sample_submission['visit_date'],format= '%Y-%m-%d %H:%M:%S')

In [3]:
air_visit_data['visit_date'] = pd.to_datetime(air_visit_data['visit_date'],format= '%Y-%m-%d')
air_visit_data.visit_date.min()

Timestamp('2016-01-01 00:00:00')

In [4]:
air_visit_data['visit_date_month'] =air_visit_data.visit_date.dt.month
air_visit_data['visit_date_dayofw'] =air_visit_data.visit_date.dt.dayofweek
air_visit_data['visit_date_year'] =air_visit_data.visit_date.dt.year
air_visit_data['visit_date_dayofm'] =air_visit_data.visit_date.dt.day
air_visit_data['weekofyear'] =air_visit_data.visit_date.dt.weekofyear

In [5]:
df=air_visit_data.groupby(['air_store_id','visit_date_month']).agg('mean')['visitors'].reset_index()

In [6]:
df.columns

Index([u'air_store_id', u'visit_date_month', u'visitors'], dtype='object')

In [7]:
df = df.pivot(index='air_store_id', columns='visit_date_month', values='visitors').reset_index()

In [8]:
df = df[['air_store_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]

In [9]:
df.columns = ['air_store_id', 'm0', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6','m7', 'm8', 'm9', 'm10', 'm11']

In [10]:
df.isnull().sum()

air_store_id      0
m0               12
m1               10
m2                1
m3                4
m4              515
m5              515
m6               30
m7               27
m8               22
m9               20
m10              18
m11              18
dtype: int64

In [11]:
df = df.drop(['m4','m5'],axis=1)

In [12]:
df.replace(np.nan,0,inplace = True)

In [13]:
df.to_csv('air_store_month_wise_mean.csv',index= False)

In [14]:
df

,air_store_id,m0,m1,m2,m3,m6,m7,m8,m9,m10,m11
0,air_00a91d42b08b08d9,25.869565,29.652174,29.384615,27.222222,26.080000,24.166667,22.833333,23.250000,22.625000,30.428571
1,air_0164b9927d20bcc3,10.789474,7.476190,9.625000,8.315789,0.000000,0.000000,0.000000,10.000000,8.260870,10.333333
2,air_0241aa3964b7f861,9.540000,9.212766,9.962264,10.250000,8.178571,10.125000,10.954545,9.153846,9.600000,12.000000
3,air_0328696196e46f18,7.714286,7.571429,9.470588,8.384615,8.000000,6.100000,10.500000,10.666667,3.888889,10.666667
4,air_034a3d5b40d5b1b1,5.733333,27.428571,23.806452,22.409091,13.777778,11.444444,9.333333,7.407407,12.272727,9.357143
5,air_036d4f1ee7285390,17.807692,16.071429,20.161290,20.590909,29.483871,32.840000,24.733333,23.451613,21.266667,17.666667
6,air_0382c794b73b51ad,13.864865,24.322581,28.333333,27.954545,22.857143,21.166667,20.241379,23.962963,27.880000,32.464286
7,air_03963426c9312048,40.722222,36.716981,45.051724,45.040816,37.038462,31.333333,34.884615,31.307692,31.370370,49.269231
8,air_04341b588bde96cd,35.543860,33.228070,41.387097,37.519231,38.806452,29.548387,35.400000,35.322581,34.200000,35.833333
9,air_049f6d5b402a31b2,10.250000,11.541667,12.214286,12.894737,11.346154,12.111111,12.000000,9.777778,10.107143,13.200000
